In [1]:
# Start by importing packages we need:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [5]:
# Set the data input directories:
current_dir =  '/Users/burl878/OneDrive - PNNL/Documents/IMMM/Data/wrf_to_tell/wrf_tell_bas_output'
cooler_dir = os.path.join(current_dir, r'rcp85cooler_ssp3')
hotter_dir = os.path.join(current_dir, r'rcp85hotter_ssp3')


In [ ]:
# Set the balancing authority to analyze:
ba_to_plot = 'CISO'

In [23]:
# Load in the raw data from the cooler scenario:
cooler_2081_df = pd.read_csv(os.path.join(cooler_dir, f'{ba_to_plot}_WRF_Hourly_Mean_Meteorology_2081.csv'), index_col=None, header=0)
cooler_2082_df = pd.read_csv(os.path.join(cooler_dir, f'{ba_to_plot}_WRF_Hourly_Mean_Meteorology_2082.csv'), index_col=None, header=0)
cooler_2083_df = pd.read_csv(os.path.join(cooler_dir, f'{ba_to_plot}_WRF_Hourly_Mean_Meteorology_2083.csv'), index_col=None, header=0)

# Concatenate the three dataframes together:
cooler_df = pd.concat([cooler_2081_df, cooler_2082_df, cooler_2083_df])

# Rename the T2 variable:
cooler_df.rename(columns={'T2': 'T2_Cooler'}, inplace=True)

# Subset to only the columns we need:
cooler_df = cooler_df[['Time_UTC', 'T2_Cooler']]

# Load in the raw data from the hotter scenario:
hotter_2081_df = pd.read_csv(os.path.join(hotter_dir, f'{ba_to_plot}_WRF_Hourly_Mean_Meteorology_2081.csv'), index_col=None, header=0)
hotter_2082_df = pd.read_csv(os.path.join(hotter_dir, f'{ba_to_plot}_WRF_Hourly_Mean_Meteorology_2082.csv'), index_col=None, header=0)
hotter_2083_df = pd.read_csv(os.path.join(hotter_dir, f'{ba_to_plot}_WRF_Hourly_Mean_Meteorology_2083.csv'), index_col=None, header=0)

# Concatenate the three dataframes together:
hotter_df = pd.concat([hotter_2081_df, hotter_2082_df, hotter_2083_df])

# Rename the T2 variable:
hotter_df.rename(columns={'T2': 'T2_Hotter'}, inplace=True)

# Subset to only the columns we need:
hotter_df = hotter_df[['Time_UTC', 'T2_Hotter']]

# Merge the two dataframes together based on a common "Time_UTC" variable:
merged_df = pd.merge(cooler_df, hotter_df, how='left', on=['Time_UTC'])

# Calculate the temperature difference:
merged_df["T2_Difference"] = (merged_df['T2_Hotter'] - merged_df['T2_Cooler']).round(2)

# Clean up what we're not using anymore:
del cooler_df, cooler_2081_df, cooler_2082_df, cooler_2083_df, hotter_df, hotter_2081_df, hotter_2082_df, hotter_2083_df

# Return the merged dataframe:
merged_df

,Time_UTC,T2_Cooler,T2_Hotter,T2_Difference
0,2081-01-01 00:00:00,290.90,292.68,1.78
1,2081-01-01 01:00:00,287.85,289.58,1.73
2,2081-01-01 02:00:00,286.99,288.68,1.69
3,2081-01-01 03:00:00,286.60,288.27,1.67
4,2081-01-01 04:00:00,286.30,287.94,1.64
...,...,...,...,...
26107,2083-12-31 19:00:00,287.35,288.86,1.51
26108,2083-12-31 20:00:00,288.10,289.63,1.53
26109,2083-12-31 21:00:00,288.52,290.03,1.51
26110,2083-12-31 22:00:00,288.64,290.12,1.48
